# ADVANCED TEXT MINING
- 본 자료는 텍스트 마이닝을 활용한 연구 및 강의를 위한 목적으로 제작되었습니다.
- 본 자료를 강의 목적으로 활용하고자 하시는 경우 꼭 아래 메일주소로 연락주세요.
- 본 자료에 대한 허가되지 않은 배포를 금지합니다.
- 강의, 저작권, 출판, 특허, 공동저자에 관련해서는 문의 바랍니다.
- **Contact : ADMIN(admin@teanaps.com)**

---

## WEEK 04-2. 동적 페이지 수집하기: 앱스토어 리뷰
- Python을 활용해 가상의 브라우저를 띄워 웹페이지에서 데이터를 크롤링하는 방법에 대해 다룹니다.

---

> **\*\*\* 주의사항 \*\*\***  
본 자료에서 설명하는 웹크롤링하는 방법은 해당 기법에 대한 이해를 돕고자하는 교육의 목적으로 사용되었으며,  
이를 활용한 대량의 무단 크롤링은 범죄에 해당할 수 있음을 알려드립니다.

### 1. 가상의 브라우저 실행하기: Chrome Driver

---

In [1]:
# Python 코드를 통해 가상의 브라우저를 띄우기 위해 selenium 패키지를 import 합니다.
import selenium
from selenium import webdriver
from selenium.webdriver.common.by import By

# selenium을 활용해 브라우저를 직접 띄우는 경우, 실제 웹서핑을 할때처럼 로딩시간이 필요합니다.
# 로딩시간 동안 대기하도록 코드를 구성하기위해 time 패키지를 import 합니다.
import time

# Python 코드를 통해 웹페이지에 정보를 요청하기 위해 BeautifulSoup, urllib 패키지를 import 합니다.
from bs4 import BeautifulSoup
import requests

In [2]:
# Chrome Driver를 호출합니다.
chrome_options = webdriver.ChromeOptions()

# 브라우저에 임의로 User-agent 옵션을 넣어 Python 코드로 접속함을 숨깁니다.
chrome_options.add_argument('--user-agent="Mozilla/5.0 (Macintosh; Intel Mac OS X 10_13_4) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/66.0.3359.139 Safari/537.36"')

# Chrome Driver 파일의 경로를 지정하고 실행합니다.
# Chrome Driver는 아래 링크에서 다운로드 가능합니다.

# 본 Jupyter Notebook 파일과 동일한 경로에 Chrome Driver가 존재하는 경우 아래 경로를 그대로 사용합니다.

# Windows 운영체제
#driver = webdriver.Chrome(executable_path = "chromedriver", chrome_options=chrome_options)

# MAC, Linux 운영체제
driver = webdriver.Chrome(executable_path = "./chromedriver", chrome_options=chrome_options)

<ipython-input-2-9c2777329be6>:16: DeprecationWarning: use options instead of chrome_options
  driver = webdriver.Chrome(executable_path = "./chromedriver", chrome_options=chrome_options)


### 2. 리뷰 페이지 이동하기

In [3]:
URL = "https://apps.apple.com/kr/app/%EC%8B%A0%ED%95%9C%ED%8E%98%EC%9D%B4%ED%8C%90/id572462317#see-all/reviews"
driver.get(URL)

### 3. 리뷰 스크롤 내리기

In [4]:
# 스크롤 횟수 (리뷰 개수에 따라 조정 필요함)
scroll_cnt = 3

for i in range(scroll_cnt):
    driver.execute_script('window.scrollTo(0, document.body.scrollHeight);')
    time.sleep(3)

In [5]:
review_box_list = driver.find_elements(By.CLASS_NAME, "we-customer-review")
print("리뷰개수 :", len(review_box_list))

리뷰개수 : 50


### 4. 리뷰 수집하기

In [6]:
f = open("shinhan.txt", "w", encoding = "utf-8")

progress = 0
for review_box in review_box_list:
    progress += 1
    print(progress, end="\r")
    
    # 리뷰별점 가져오기
    review_star = review_box.find_element(By.CLASS_NAME, "we-star-rating-stars")
    review_star_count = review_star.get_attribute("class")[-1]
    
    # 더보기 버튼 클릭하기: try-except 예외처리 적용
    try:
        review_box.find_element(By.CLASS_NAME, "we-truncate__button").click()
        # 리뷰 텍스트 가져오기
        review_text = driver.find_element(By.CLASS_NAME, "we-customer-review__body--modal").text
        review_text = review_text.replace("\n", " ").strip()
        # 팝업 레이어 닫기
        driver.find_element(By.CLASS_NAME, "we-modal__close").click()
    # 더보기 버튼이 없은 경우 Exception 처리하기
    except:
        review_text = review_box.find_element(By.CLASS_NAME, "we-clamp").text
        review_text = review_text.replace("\n", " ").strip()
        
    # 파일에 쓰기
    f.write(review_star_count + "\t" + review_text + "\n")
f.close()

In [7]:
f = open("shinhan.txt", encoding = "utf-8")

for line in f:
    print(line.strip())
    
f.close()

2	제가 토스카드 이벤트(현금지원)때문에 이 카드 저카드 많이 써보거든요?  솔직히 현대카드가 UI나 사용편의성이 상당히 좋습니다 벤치마킹좀 했으면 좋겠어요
5	최근에 업데이트 된 것 같은데 앱을 켜고 택시 페이를 선택하면 다음 화면(결제 정보를 수신하고 있다...)으로 넘어가지 않아서 사용하지 않다가 요즘 잘 되네요. 속도도 빠르고 몇 번 했도 문제없이 결제까지 잘 되네요...저만 사용하는 건 아니죠? 임산부나 카드 내기 귀찮으신분들 워해 많이 홍보해서 이용하게 하면 좋을 것 같습니다!
5	그냥 미친어플 입니다.. 너무 편해서 그냥 계속 사요,, 차라리 번거롭게 카드번호 치고 어쩌고 하면 더 아낄 수 있을 텐데.........
5	유용하게 잘 사용하고 있습니다 아이폰 xr 사용자이고 face id가 되어 매우 편리하고 좋습니다 감사합니다 질문할 것이 있는데 새 알림이 있다고 앱아이콘에 숫자가 뜨는데 앱에 들어가 보면 없어요 그냥 홈화면으로 이용대금이 나오고 타임라인, 홈, 서비스 어디에도 앱 아이콘에 뜬 숫자가 무얼 말하는건지 모르겠어요 새로 결제한게 있다는 거겠죠?
4	현재 우리나라에서 애플페이 미지원으로 신한카드를 쓰는 아이폰 이용자 대부분이 fan앱카드나 실물카드로 결제를 대신하고 있는데 저역시 실물카드를 항상 소지하고 다니며 그나마 오프라인 가맹점에서만 간간히 앱카드 내의 바코드 생성으로 결제를 이용하고 있는데요.. 정말 편한 기능이고 잘 쓰고 있어요! 근데 실제 현장에서 앱카드로 결제를 할때 어플을 켜고 인증 받고 바코드 표시까지의 시간이 꽤 오래걸려서 계산대에서 눈치도 보이고.. 저는 애초에 매장에 들어갈 때 앱카드부터 켜고 들어가요 ㅠ 저를 포함해 아마 많은분들이 애플페이 대신 마이애플월렛을 통해 애플월렛에 카드를 등록해서 쓰고 있는데 신한카드도 월렛에 등록해서 바로바로 꺼내 쓸 수 있다면 정말 좋을것같아요.. 방법을 찾아.. 아니 만들어주시면 너무너무 감사할듯해요!😭😭
5	리뷰에 별로라는 말이 너무많아서 걱정했는데 굉장히 편하고 잘되어있습니다.

### Assignment 03. 동적 페이지 수집하기: 플레이스토어 리뷰
> - 본 자료를 응용하여 구글 플레이스토어 앱 리뷰를 대량으로 가져오는 코드를 작성합니다.
> - 스크롤을 충분히 내려 리뷰를 500개 이상 수집합니다.
> - 관리자 댓글은 무시하고 고객 리뷰만 수집합니다.
> - 리뷰 텍스트와 함께 작성일자와 별점을 같이 가져옵니다.
> - 리뷰, 작성일자, 별점을 탭(\t) 단위로 구분하여 파일에 저장합니다.  

---